In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time

from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier

In [2]:
startTime = time.time()

fileName = "Basketball Data/Play_by_Play_New.csv"
df = pd.read_csv(fileName)

playerMatch = "Basketball Data/Player_Map.csv"
player_df = pd.read_csv(playerMatch)

teamMatch = "Basketball Data/Team_Map.csv"
player_df = pd.read_csv(teamMatch)

endTime = time.time()
print("Time Elapsed:", endTime - startTime, "sec")

Time Elapsed: 28.78437042236328 sec


In [3]:
startIndex = np.where(df["Season"] == 2016)[0][0]

# keep only 2016 data
print("Headers:")
for header in df:
    print(header)
    df[header] = df[header][startIndex:]
print()

Headers:
Game_id
Season
Season_Type
Game_No
Playoff_Rd
Playoff_Rd_Game_No
Date
Home_Team_id
Home_Tm
Visitor_Team_id
Away_Tm
Period
Event_Num
Wall_Clock_Time
Play_Clock_Time
Team_Committing_Action
Person1
Person2
Person3
Home_PTS
Visitor_PTS
X_Location
Y_Location
Description
Rebound_Designation
Shot_Value
Shot_Outcome
Shot_Side_of_Ct
Shot_Distance
General_Description
Player1
Player2
Player3
Player4
Player5
Player6
Player7
Player8
Player9
Player10



In [4]:
# keep only shot data
df = df[np.isfinite(df['Shot_Value'])]

# keep only clutch time data
df = df[np.isfinite(df['Play_Clock_Time'])]
df = df.loc[df['Play_Clock_Time'] <= 120]

df['Score_Difference'] = df['Home_PTS'] - df['Visitor_PTS']
df['Score_Difference'] = df['Score_Difference'].abs()

# keep only clutch situation data
df = df.loc[df['Score_Difference'] <= 10]

In [8]:
X = df[['Wall_Clock_Time','Shot_Distance', 'Shot_Side_of_Ct', 'X_Location']]
Y = df["Shot_Value"]

In [10]:
# Normalize Data
# xMean, xStd = X.mean(), X.std()
# xNormal = (X - xMean) / xStd

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

bestScore = 0
bestModel = None
bestCombo = (8, 11)

"""
print("Grid Searching for Optimal # of Layers / Units...")
for layers in range(2, 15):
    for units in range(2, 15):
        model = MLPClassifier(hidden_layer_sizes=(layers, units), alpha=0.001, random_state=20, activation="logistic",
                              batch_size='auto', learning_rate='constant', learning_rate_init=0.01)
        model.fit(X_train, y_train.ravel())
        currentScore = model.score(X_test, y_test.ravel())
        print(str(layers) + " layers & " + str(units) + " units yields score =", str(currentScore))
        if currentScore > bestScore:
            bestScore = currentScore
            bestModel = model
            bestCombo = (layers, units)
"""

# Parameters from Previous Grid Searching
bestModel = MLPClassifier(hidden_layer_sizes=bestCombo, alpha=0.001, random_state=20,
                     batch_size='auto', learning_rate='constant', learning_rate_init=0.01)
bestModel.fit(X, Y.ravel())
predicted_Y = bestModel.predict(X)


print("Done!")
print()

print("Accuracy =", bestModel.score(X_test, y_test.ravel()) * 100, "%")
print("Layers / Units:", bestCombo)
print()

ValueError: could not convert string to float: 'LC'